# <B> Evaluation </B>
* Container: codna_pytorch_py39

## AutoReload

In [6]:
import boto3

## 1. parameter store 설정

In [25]:
!df -h

Filesystem         Size  Used Avail Use% Mounted on
overlay             27G  384K   27G   1% /
tmpfs               64M     0   64M   0% /dev
tmpfs              1.9G     0  1.9G   0% /sys/fs/cgroup
shm                397M     0  397M   0% /dev/shm
127.0.0.1:/200005  8.0E  356M  8.0E   1% /root
/dev/nvme0n1p1     140G   24G  117G  17% /opt/.sagemakerinternal
devtmpfs           1.9G     0  1.9G   0% /dev/tty
tmpfs              1.9G     0  1.9G   0% /proc/acpi
tmpfs              1.9G     0  1.9G   0% /sys/firmware


In [7]:
import os
import boto3

class parameter_store():
    
    def __init__(self, region_name="ap-northeast-2"):
        
        self.ssm = boto3.client('ssm', region_name=region_name)
        
    def put_params(self, key, value, dtype="String", overwrite=False, enc=False) -> str:
        
        #aws ssm put-parameter --name "RDS-MASTER-PASSWORD" --value 'PASSWORD' --type "SecureString"
        if enc: dtype="SecureString"
        if overwrite:
            strQuery = ''.join(['aws ssm put-parameter', ' --name ', '"', str(key), '"', ' --value ', '"', str(value), '"', ' --type ', '"', str(dtype), '"', ' --overwrite'])
        strResponse = os.popen(strQuery).read()
        
        if strResponse != '': return 'Store suceess'
        else: return 'Error'
    
    def get_params(self, key, enc=False):
        
        if enc: WithDecryption = True
        else: WithDecryption = False
        response = self.ssm.get_parameters(
            Names=[key,],
            WithDecryption=WithDecryption
        )
        
        return response['Parameters'][0]['Value']

    def get_all_params(self, ):

        response = self.ssm.describe_parameters(MaxResults=50)

        return [dicParam["Name"] for dicParam in response["Parameters"]]

    def delete_param(self, listParams):

        response = self.ssm.delete_parameters(
            Names=listParams
        )
        print (f"  parameters: {listParams} is deleted successfully")

In [8]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
prefix = pm.get_params(key="PREFIX")

## 2.Processing-job for evaluation

In [9]:
import os
import sagemaker
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor

* **Set Up SageMaker Experiment**
    - Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

* params for processing job

In [10]:
!pip install sagemaker-experiments

In [11]:
from time import strftime
from smexperiments.trial import Trial
from smexperiments.experiment import Experiment

In [12]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [13]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

In [14]:
select_date = '2023/03/20'
variant_name = 'AllTraffic'


In [17]:
local_mode = False

if local_mode: 
    instance_type = 'local_gpu'
    
    import os
    from sagemaker.local import LocalSession
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
else:
    instance_type = "ml.g4dn.xlarge"
    sagemaker_session = sagemaker.Session()


sagemaker_role_arn = pm.get_params(key=prefix + '-SAGEMAKER-ROLE-ARN')
preprocessing_data_path = pm.get_params(key=prefix + '-PREP-DATA-PATH')
bucket_name = pm.get_params(key=prefix + '-BUCKET')
eval_image_uri = pm.get_params(key=''.join([prefix, '-IMAGE-URI']))
model_artifact_s3_uri = pm.get_params(key=prefix + '-MODEL-PATH')
test_manifest_s3_path = os.path.join(preprocessing_data_path, "an4", "test_manifest.json")
wav_s3_uri = os.path.join(preprocessing_data_path, "an4", "wav")
eval_output_s3_uri = os.path.join("s3://", bucket_name, "evaluation", "output", "eval_output")
endpoint_name = pm.get_params(key=prefix + '-ENDPOINT-NAME')
monitor_output = pm.get_params(key=''.join([prefix, '-MONITOR-OUTPUT']))

code_location= os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "evaluation",
    "backup_codes"
)

#s3://sagemaker-us-west-2-322537213286/nemo-prod/inference/monitor_output
#s3://sagemaker-us-west-2-322537213286/nemo-prod/inference/monitor_output/nemo-prod-nemo-experiments-0320-07331679297605/AllTraffic/2023/03/20/07/

inference_output_s3uri = os.path.join(
    monitor_output,
    endpoint_name,
    variant_name,
    select_date
)

gtmanifest_s3uri = os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "gt-manifest"
)

output_s3uri = os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "pred-output"
)

print (f"instance-type: {instance_type}")
print (f"image-uri: {eval_image_uri}")
print (f"role: {sagemaker_role_arn}")
print (f"bucket: {bucket_name}")

print (f"model_artifact_s3_uri: {model_artifact_s3_uri}")
print (f"test_manifest_s3_path: {test_manifest_s3_path}")
print (f"wav_s3_uri: {wav_s3_uri}")
print (f"eval_output_s3_uri: {eval_output_s3_uri}")
print (f"endpoint_name: {endpoint_name}")
print (f"monitor_output: {monitor_output}")
print (f"sagemaker_session: {sagemaker_session}")

instance-type: ml.g4dn.xlarge
image-uri: 322537213286.dkr.ecr.us-west-2.amazonaws.com/nemo-test-training:latest
role: arn:aws:iam::322537213286:role/service-role/AmazonSageMaker-ExecutionRole-20210401T133000
bucket: sagemaker-us-west-2-322537213286
model_artifact_s3_uri: s3://sagemaker-us-west-2-322537213286/nemo-prod/training/model-output/nemo-prod-nemo-experiments-0320-07471679298425/output/model.tar.gz
test_manifest_s3_path: s3://sagemaker-us-west-2-322537213286/preprocessing/data/an4/test_manifest.json
wav_s3_uri: s3://sagemaker-us-west-2-322537213286/preprocessing/data/an4/wav
eval_output_s3_uri: s3://sagemaker-us-west-2-322537213286/evaluation/output/eval_output
endpoint_name: nemo-prod-nemo-experiments-0320-07331679297605
monitor_output: s3://sagemaker-us-west-2-322537213286/nemo-prod/inference/monitor_output
sagemaker_session: <sagemaker.session.Session object at 0x7f9b1ca21550>


* Define processing job

In [18]:
pred_processor = FrameworkProcessor(
    estimator_cls=PyTorch,
    framework_version="1.13.1",
    role=sagemaker_role_arn, 
    image_uri=eval_image_uri,
    instance_count=1,
    instance_type=instance_type,
    code_location=code_location,
    env={
        'select_date': select_date, 
        'tolerance' : "0.5"
        }
    )

In [19]:
gt_list = ['rubout n   she yn', 'rubout n   she yn','rubout n   she yn','rubout n   she yn','rubout n   she yn','rubout n   she yn','rubout n   she yn','rubout n   she yn']
len(gt_list)

8

In [20]:
import pathlib
pathlib.Path("manifest").mkdir(parents=True, exist_ok=True)

In [21]:
import pickle

with open('manifest/gt_manifest.pkl', 'wb') as f:
    pickle.dump(gt_list, f, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
!aws s3 sync ./manifest $gtmanifest_s3uri

upload: manifest/gt_manifest.pkl to s3://sagemaker-us-west-2-322537213286/nemo-prod/gt-manifest/gt_manifest.pkl


In [23]:
experiment_name = pm.get_params(key=prefix + "-EXPERI-NAME")
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

pred_processor.run(
    code="evaluate.py",
    source_dir=os.getcwd() + "/code",
    inputs=[
        ProcessingInput(
            source=inference_output_s3uri, 
            input_name="inference_data", 
            destination="/opt/ml/processing/input/inference_data"
        ),
        ProcessingInput(
            source=gtmanifest_s3uri, 
            input_name="gt_manifest", 
            destination="/opt/ml/processing/input/manifest"),
    ],
    outputs=[
        ProcessingOutput(
            source=f"/opt/ml/processing/output/{select_date}/manifest", 
            destination=output_s3uri + '/manifest'
        ),
        ProcessingOutput(
            source=f"/opt/ml/processing/output/{select_date}/wav", 
            destination=output_s3uri + '/wav'
        ),
    ],
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=True
)

INFO:sagemaker.processing:Uploaded /root/nemo-on-sagemaker/2 building-pipelines/code to s3://sagemaker-us-west-2-322537213286/nemo-prod/evaluation/backup_codes/nemo-prod-nemo-experiments-0320-14451679323547/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-west-2-322537213286/nemo-prod/evaluation/backup_codes/nemo-prod-nemo-experiments-0320-14451679323547/source/runproc.sh
INFO:sagemaker:Creating processing-job with name nemo-prod-nemo-experiments-0320-14451679323547



Job Name:  nemo-prod-nemo-experiments-0320-14451679323547
Inputs:  [{'InputName': 'inference_data', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-322537213286/nemo-prod/inference/monitor_output/nemo-prod-nemo-experiments-0320-07331679297605/AllTraffic/2023/03/20', 'LocalPath': '/opt/ml/processing/input/inference_data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'gt_manifest', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-322537213286/nemo-prod/gt-manifest', 'LocalPath': '/opt/ml/processing/input/manifest', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-322537213286/nemo-prod/evaluation/backup_codes/nemo-prod-nemo-experiments-0320-14451679323547/source/sourcedir.tar.gz', 'LocalPath':

UnexpectedStatusException: Error for Processing job nemo-prod-nemo-experiments-0320-14451679323547: Failed. Reason: AlgorithmError: See job logs for more information